In [79]:
import requests
import io
import pandas as pd
import numpy as np
import pandas_ta as ta
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split,GridSearchCV
from sklearn.metrics import plot_confusion_matrix, plot_roc_curve, accuracy_score, recall_score, precision_score, f1_score
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFECV
from xgboost import XGBClassifier

from os import listdir
from os.path import isfile, join

import warnings
warnings.filterwarnings("ignore")

In [80]:
files = [f for f in listdir('./S&P500_data')]

In [81]:
companies = []
for i in files:
    company = i.split('.')[0]
    companies.append(company)
    
companies.pop(0)

''

In [82]:
total_df = pd.DataFrame()

for x in companies:
    csv_for_df = f'S&P500_data/{x}.csv'
    df = pd.read_csv(csv_for_df)
    df['name'] = x
    total_df = total_df.append(df, ignore_index = True, sort = False)

In [83]:
total_df = total_df[['Date','Adjusted Close', 'name']]

In [84]:
total_df['Date'] = pd.to_datetime(total_df.Date)
total_df.set_index('Date', inplace = True)

In [85]:
total_df

,Adjusted Close,name
Date,,
1999-11-18,27.102230,A
1999-11-19,24.869375,A
1999-11-22,27.102230,A
1999-11-23,24.638390,A
1999-11-24,25.292850,A
...,...,...
2021-12-14,228.070007,ZTS
2021-12-15,232.080002,ZTS
2021-12-16,232.809998,ZTS


In [86]:
total_df = total_df.pivot(columns = 'name', values = 'Adjusted Close')

In [87]:
total_df.applymap(lambda x: np.log(x))

name,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADI,...,WYNN,XEL,XLEFF,XLNX,XOM,XYL,YUM,ZBH,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
1970-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-1.809015,NaN,NaN,NaN,NaN,NaN
1970-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-1.871816,NaN,NaN,NaN,NaN,NaN
1970-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-1.803254,NaN,NaN,NaN,NaN,NaN
1970-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-1.843618,NaN,NaN,NaN,NaN,NaN
1970-01-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-1.651202,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-14,5.023091,2.827905,5.457053,5.160950,4.846468,4.814620,5.740661,4.889898,5.912773,5.172074,...,4.422689,4.210497,-1.442653,5.346202,4.119687,4.776431,4.882726,4.814458,4.131480,5.429653
2021-12-15,5.038964,2.832625,5.472859,5.189060,4.869302,4.818425,5.754032,4.909414,5.927726,5.190342,...,4.415341,4.231494,-1.442653,5.383256,4.115290,4.785406,4.885979,4.793142,4.139318,5.447082
2021-12-16,5.015954,2.804572,5.477133,5.149005,4.881134,4.820120,5.736572,4.913316,5.992963,5.150977,...,4.389002,4.233382,-1.442653,5.297217,4.116921,4.776094,4.894775,4.791401,4.150567,5.450223


In [88]:
p_change = total_df.diff()

In [89]:
median = p_change.quantile(.5, axis = 1)

In [90]:
ninety = p_change.quantile(.90, axis = 1)

In [91]:
ten = p_change.quantile(.1, axis = 1)

In [92]:
market_sentiment = pd.concat([ten,median,ninety], axis = 1)

In [93]:
market_sentiment = market_sentiment.applymap(lambda x: np.exp(x))

In [94]:
market_sentiment

,0.1,0.5,0.9
Date,,,
1970-01-04,NaN,NaN,NaN
1970-01-05,0.858933,0.963959,0.990372
1970-01-06,0.921497,0.982907,1.004322
1970-01-07,0.861120,0.979124,1.006331
1970-01-09,0.989002,1.024405,1.186861
...,...,...,...
2021-12-14,0.007704,0.763376,2.767652
2021-12-15,0.726149,2.127573,709.098068
2021-12-16,0.006738,1.000000,8.593476


In [95]:
df['date'] = pd.to_datetime(df.date, format = '%d-%m-%Y')
df.set_index('date', inplace = True)
df.sort_index(inplace = True)

AttributeError: 'DataFrame' object has no attribute 'date'